### Importing the libraries and Configuring the API Key

In [1]:
import pandas as pd
import numpy as np
import google.generativeai as genai
from dotenv import dotenv_values

In [2]:
df = pd.read_csv('dataset/tmdb_5000_movies_embed.csv')

In [3]:
df.head(5)

,title,overview,Embeddings
0,Avatar,"In the 22nd century, a paraplegic Marine is di...","[0.031759717, 0.0044186483, -0.025144888, -0.0..."
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[0.06617279, 0.008563878, -0.04165544, -0.0447..."
2,Spectre,A cryptic message from Bond’s past sends him o...,"[-0.012871856, -0.005491358, 0.012047383, -0.0..."
3,The Dark Knight Rises,Following the death of District Attorney Harve...,"[0.04840929, 0.03056959, -0.030966783, -0.0182..."
4,John Carter,"John Carter is a war-weary, former military ca...","[0.061523866, -0.032111533, -0.035515673, 0.00..."


In [4]:
df['Embeddings'] = df['Embeddings'].apply(lambda x: eval(x))

In [5]:
print(type(df['Embeddings'][0]))

<class 'list'>


In [6]:
ENV = dotenv_values(".env")
api_key = ENV['API_KEY']
genai.configure(api_key=api_key)

### Creating the Embedding function to the query

In [7]:
embedding_model = "models/embedding-001"

In [8]:
def embed_content(title, query, model, task_type):
    if query is None:
        return genai.embed_content(
            model=model,
            content=query,
            task_type=task_type,
        )["embedding"]
    else:
        return genai.embed_content(
            model=model,
            title=title,
            content=query,
            task_type=task_type,
        )["embedding"]

In [9]:
def get_suggestion(query, df, model):
    query_embedding = embed_content(
        None,
        query,
        model,
        "RETRIEVAL_QUERY",
    )
    
    # df['Embeddings'] = df['Embeddings'].apply(lambda x: np.array(x))
    scalars = np.dot(np.stack(df["Embeddings"]), query_embedding, out=None)
    index = np.argmax(scalars)
    return df.iloc[index][["title", "overview"]]

### Query of the user

In [10]:
query = "A movie about white rapper"
suggestion = get_suggestion(query, df, embedding_model)
print(f"Title: {suggestion.title}")
print(f"Description: {suggestion.overview}")

Title: 8 Mile
Description: The setting is Detroit in 1995. The city is divided by 8 Mile, a road that splits the town in half along racial lines. A young white rapper, Jimmy "B-Rabbit" Smith Jr. summons strength within himself to cross over these arbitrary boundaries to fulfill his dream of success in hip hop. With his pal Future and the three one third in place, all he has to do is not choke.


### Configuring the response from Gemini

In [16]:
generation_config = {
  "temperature": 0.5,
  "top_k": 50,
  "top_p": 0.95,
  "candidate_count": 1,
}

In [12]:
safety_settings = {
    'HARASSMENT': 'BLOCK_NONE',
    'HATE': 'BLOCK_NONE',
    'SEXUAL': 'BLOCK_NONE',
    'DANGEROUS': 'BLOCK_NONE',
}

In [13]:
prompt = f"Com base na [QUERY] sobre características/atributos de filmes: {query}. Elabore um breve resumo de um parágrafo para o filme: {suggestion.title}, considerando as características citadas na [QUERY]. Seja simpático e divertido, e busco me convencer a assistir a esse filme"

In [17]:
model = genai.GenerativeModel(
    model_name='gemini-1.0-pro',
    generation_config=generation_config, 
    safety_settings=safety_settings
)

In [18]:
response = model.generate_content(prompt)

### Response

In [20]:
print(response.text)

Prepare-se para um passeio eletrizante pelas ruas de Detroit em "8 Mile", um filme que gira em torno do aspirante a rapper branco Jimmy "B-Rabbit" Smith Jr. Lutando contra a pobreza e o preconceito, B-Rabbit encontra refúgio na música, usando sua voz para expressar suas lutas e sonhos. Com rimas afiadas e um coração de ouro, ele enfrenta batalhas de rap intensas, provando que a cor da pele não pode definir seu talento. Testemunhe a jornada inspiradora de B-Rabbit enquanto ele supera obstáculos, encontra seu lugar no mundo do hip-hop e deixa sua marca na indústria da música.


### Chatbot

In [21]:
import textwrap
from IPython.display import display, Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [22]:
query = ""
while query != "exit":
    query = input("Write a query: ")
    if query != "exit":
        suggestion = get_suggestion(query, df, embedding_model)
        ## apply the function markdown to title and description
        display(to_markdown(f"Usuário: {query}"))
        display(to_markdown(f"BOT: Filme: {suggestion.title}"))
        prompt = f"Com base na [QUERY] sobre características/atributos de filmes: {query}. Elabore um breve resumo de um parágrafo para o filme: {suggestion.title}, considerando as características citadas na [QUERY]. Seja simpático e divertido, e busco me convencer a assistir a esse filme"
        response = model.generate_content(prompt)
        display(to_markdown(f'BOT: {response.text}'))

> Usuário: A movie about a rapper

> BOT: Filme: Get Rich or Die Tryin'

> BOT: Prepare-se para uma jornada musical eletrizante com "Get Rich or Die Tryin'"! Siga a história de Marcus, um talentoso rapper que luta para sobreviver nas ruas de Nova York. Com rimas afiadas e batidas pulsantes, ele sonha em escapar da pobreza e alcançar o estrelato.
> 
> Mas o caminho para o sucesso é repleto de obstáculos. Marcus enfrenta gangues rivais, traições e um passado sombrio que o assombra. No entanto, sua determinação e paixão pela música o mantêm motivado.
> 
> Com um elenco estelar, incluindo 50 Cent no papel principal, "Get Rich or Die Tryin'" é uma história inspiradora sobre a busca implacável dos sonhos, o poder da música e a resiliência do espírito humano. Prepare-se para ser cativado por essa jornada musical emocionante e motivadora!

> Usuário: A movie about a white rapper

> BOT: Filme: 8 Mile

...

> BOT: Filme: Super

> BOT: Prepare-se para uma aventura épica com "Super", um filme que redefine o conceito de super-herói!
> 
> Imagine um homem comum chamado Frank, que decide se tornar o "Crimson Bolt" para proteger sua cidade do mal. Com um traje caseiro e uma determinação inabalável, Frank embarca em uma missão para salvar o mundo de um vilão excêntrico e perigoso.
> 
> Ao longo do caminho, Frank encontra uma aliada improvável em Libby, uma aspirante a super-heroína que compartilha sua paixão por justiça. Juntos, eles enfrentam perigos, desvendam segredos e provam que até mesmo os heróis mais improváveis podem fazer a diferença.
> 
> "Super" é uma comédia de ação hilária e comovente que celebra o poder da imaginação e o espírito indomável do homem comum. Com personagens cativantes, ação emocionante e uma mensagem inspiradora, este filme certamente o deixará entretido e motivado.

> Usuário: A movie about a musician falling in love

> BOT: Filme: Mutual Appreciation

> BOT: Prepare-se para uma sinfonia de romance e música em "Mutual Appreciation"!
> 
> Este filme é uma obra-prima sobre o poder transformador do amor e da arte. Acompanhe a jornada de um talentoso músico que se apaixona perdidamente por uma mulher que o inspira a criar sua música mais comovente.
> 
> Entre notas musicais e olhares apaixonados, eles descobrem que o amor pode ser o ritmo que une seus corações. Mas o caminho para a harmonia não é sem seus desafios.
> 
> Com personagens cativantes, uma trilha sonora inesquecível e uma história que vai aquecer seu coração, "Mutual Appreciation" é um filme que vai fazer você dançar de alegria e cantarolar de amor. Não perca essa obra-prima cinematográfica que prova que o amor e a música são os ingredientes perfeitos para uma sinfonia inesquecível.